In [ ]:
#@title mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title load relevant packages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from time import strptime
from scipy import stats
import zipfile
import json
import os

In [ ]:
#@title make .pkl file
path = '/content/drive/MyDrive/insight_data_analysis/Insight4wear Dataset 24 May 2021/dataset/final_data'
users_list = []
user_activity_list = []
for root, directories, files in os.walk(path):
  for user in directories:
      folder = path + '/' + user
      for root, directories, files in os.walk(folder):
        for file in files:
          if file.startswith('w') and file.endswith('.zip'):
            zip = folder + '/' + file
            try:
                with zipfile.ZipFile(zip) as z:
                    folder_list = z.namelist()
                    activity_file = [x for x in folder_list if x.startswith('Activity/')]
                    if len(activity_file) == 1:
                        activity_file = activity_file[0]
                        for i in range(int(np.floor((len(z.open(activity_file).read()))/len((z.open(activity_file)).readlines())))):
                            with z.open(activity_file) as f:
                                data = f.readlines()[i]
                                data = json.loads(data)
                                lst = [user, data['sensor_name'], data['timestamp'],
                                        data['time_stamp'], data['sensor_data']['step_counts'],
                                        data['sensor_data']['step_delta']]
                                user_activity_list.append(lst)

            except Exception as e:
                print(e)

user_activity_df = pd.DataFrame(user_activity_list,
                                columns = ['user', 'sensor_name', 'timestamp',
                                           'time_stamp', 'step_counts', 'step_delta'])


In [ ]:
%cd /content/drive/MyDrive/insight_data_analysis/Insight4wear Dataset 24 May 2021/result_pkl_file/pkl_files

user_activity_df = pd.DataFrame(user_activity_list,
                                columns = ['user', 'sensor_name', 'timestamp',
                                           'time_stamp', 'step_counts', 'step_delta'])

user_activity_df.to_pickle('./Activity.pkl')
Activity = pd.read_pickle(r'Activity.pkl')

In [ ]:
df.info()
df.describe()

In [ ]:
df['day-name'].value_counts()
df['month'].value_counts()
df['weekend'].value_counts()
df['covid'].value_counts()
df['time'].value_counts()

In [ ]:
#@title vizualization
import seaborn as sb
for i in range(0,9):
  sb.catplot(x=df.columns[i], y="step_counts", kind="bar", data=df)
  plt.xlabel('activity in the morning', fontsize=15)
  plt.xticks(rotation = 90)


In [ ]:
#@title vizualization
import seaborn as sb
plt.rcParams['figure.figsize'] = (18, 7)

plt.subplot(1, 2, 1)

sb.countplot(dff['time'], palette='pastel')
plt.title('heart rate  ', fontsize=20)
plt.xlabel('morning               afternoon                     night', fontsize=15)
plt.ylabel('number of users', fontsize=15)

In [ ]:
#@title heatmap
newdf = dff
# newdf.drop(newdf.columns[0], inplace=True, axis=1)


corr = newdf.corr().round(2)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.rcParams['figure.figsize'] = (18, 7)
with sb.axes_style("white"):
    fig, ax = plt.subplots(figsize=(13,13))
    sb.heatmap(ax=ax, data=corr, mask=mask, annot=True, center=0, vmin=-1, vmax=1, cmap="RdBu_r")
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = active.groupby(['user', 'weekend'])['step_counts'].max().reset_index()
df_copy = data.drop(['user'], axis = 1)

x= df[df['weekend']==0]
y = df[df['weekend']==1]

ypoints = np.array([ 0,1.2e6])
xpoints = np.array([ 0,1.2e6])

plt.scatter(x, y, c='green')
plt.plot(ypoints,xpoints,color = 'r')

# plt.gca().invert_yaxis()
plt.show()

clustring

In [ ]:
#@title optics
from sklearn.datasets import make_blobs
from sklearn.cluster import OPTICS
import numpy as np
import matplotlib.pyplot as plt

X = df_copy

# Configuration options
num_samples_total = 1000
cluster_centers = [(3,3), (7,7)]
num_classes = len(cluster_centers)
epsilon =  1000000
min_samples = 35
cluster_method = 'xi'
metric = 'minkowski'

# Generate data
# X, y = make_blobs(n_samples = num_samples_total, centers = cluster_centers, n_features = num_classes, center_box=(0, 1), cluster_std = 0.5)

# Compute OPTICS
db = OPTICS(max_eps=epsilon, min_samples=min_samples, cluster_method=cluster_method, metric=metric).fit(X)
labels = db.labels_

no_clusters = len(np.unique(labels) )
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

# Generate scatter plot for training data
colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', labels))
plt.scatter(X[:,0], X[:,1], c=colors, marker="o", picker=True)
plt.title(f'OPTICS clustering')
plt.xlabel('Axis X[0]')
plt.ylabel('Axis X[1]')
plt.show()

In [ ]:
#@title K-meas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

X = df_copy

kmeans = KMeans(n_clusters = 2, max_iter = 600, random_state = 0)
label = kmeans.fit_predict(X)
print(label)
XX= kmeans.fit(X)


#Getting unique labels

u_labels = np.unique(label)

#plotting the results:


for i in u_labels:
    plt.scatter(X[label == i , 0] , X[label == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
X = pd.DataFrame(X)
X1 = X[kmeans.labels_== 0]
X1['compare'] = 0
for index, row in X1.iterrows():
    # print('Processed Row: ' + str(index))
    # print(X.loc[index,1])
    if X1.loc[index,1] >= X1.loc[index,0]:
       X1.loc[index,'compare'] = 1

print(X1['compare'].sum()/179)


X2 = X[kmeans.labels_==1]
X2['compare'] = 0
for index, row in X2.iterrows():
    # print('Processed Row: ' + str(index))
    # print(X.loc[index,1])
    if X2.loc[index,1] >= X2.loc[index,0]:
       X2.loc[index,'compare'] = 1
print(X2['compare'].sum()/125)

In [ ]:
#@title DBscan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN

X_principal = df_copy
X_principal = pd.DataFrame(X_principal)
X_principal.columns = ['P1', 'P2']

db_default = DBSCAN(eps = 180000, min_samples =80).fit(X_principal)
labels = db_default.labels_

# print(labels)
# Building the label to colour mapping
colours = {}
colours[0] = 'r'
colours[1] = 'g'
colours[2] = 'b'
colours[-1] = 'k'

# Building the colour vector for each data point
cvec = [colours[label] for label in labels]

# For the construction of the legend of the plot
r = plt.scatter(X_principal['P1'], X_principal['P2'], color ='r');
g = plt.scatter(X_principal['P1'], X_principal['P2'], color ='g');
b = plt.scatter(X_principal['P1'], X_principal['P2'], color ='b');
k = plt.scatter(X_principal['P1'], X_principal['P2'], color ='k');

# Plotting P1 on the X-Axis and P2 on the Y-Axis
# according to the colour vector defined
# plt.figure(figsize =(9, 9))
plt.scatter(X_principal['P1'], X_principal['P2'], c = cvec)

# Building the legend
plt.legend((r, g, b, k), ('Label 0', 'Label 1', 'Label 2', 'Label -1'))

plt.show()


In [ ]:
#@title Agglomerative
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering

X = df_copy
clustering = AgglomerativeClustering( )
labell = clustering.fit_predict(X)
# print(label)
clustering.fit(X)


#Getting unique labels

u_labels = np.unique(labell)

#plotting the results:

for i in u_labels:
    plt.scatter(X[label == i , 0] , X[label == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
#@title SpectralClustering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import SpectralClustering

X = df_copy
cluster = SpectralClustering(n_clusters= 8 , assign_labels='discretize',random_state=0).fit(X)
labelll = clustering.fit_predict(X)
print(label)


#Getting unique labels

u_labels = np.unique(labelll)

#plotting the results:

for i in u_labels:
    plt.scatter(X[label == i , 0] , X[label == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
#@title clustring "time"
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler

import numpy as np
import matplotlib.pyplot as plt

# Though the following import is not directly being used, it is required
# for 3D projection to work
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
# from sklearn import datasets

# np.random.seed(5)

# iris = datasets.load_iris()
# X = iris.data
# y = iris.target
print(X.shape)
estimators = [
    ("k_means_iris_8", KMeans(n_clusters=8)),
    ("k_means_iris_3", KMeans(n_clusters=3)),
    ("k_means_iris_bad_init", KMeans(n_clusters=3, n_init=1, init="random")),
]

fignum = 1
titles = ["8 clusters", "3 clusters", "3 clusters, bad initialization"]
for name, est in estimators:
    fig = plt.figure(fignum, figsize=(4, 3))
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
    est.fit(X)
    labels = est.labels_

    ax.scatter(X[:, 2], X[:, 0], X[:, 1], c=labels.astype(float), edgecolor="k")

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel("time 0")
    ax.set_ylabel("time 1")
    ax.set_zlabel("time 2")
    ax.set_title(titles[fignum - 1])
    ax.dist = 12
    fignum = fignum + 1

# Plot the ground truth
fig = plt.figure(fignum, figsize=(4, 3))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)

for name, label in [("Setosa", 0), ("Versicolour", 1), ("Virginica", 2)]:
    ax.text3D(
        X[y == label, 2].mean(),
        X[y == label, 0].mean(),
        X[y == label, 1].mean(),
        name,
        horizontalalignment="center",
        bbox=dict(alpha=0.2, edgecolor="w", facecolor="w"),
    )
# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 2, 0]).astype(float)
ax.scatter(X[:,2], X[:, 0], X[:, 1], c=y, edgecolor="k")

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel("time 0")
ax.set_ylabel("time 1")
ax.set_zlabel("time 2")
ax.set_title("Ground Truth")
ax.dist = 12

fig.show()